In [42]:
# Nhập thư viện
import numpy as np
import pandas as pd

In [43]:
# Load dữ liệu
def loadExcel(filename) -> pd.DataFrame:
    try:
        data = pd.read_excel("/content/Iris.xls")
        return data
    except Exception as e:
        print(f"Đã xảy ra lỗi khi tải file: {e}")
        return pd.DataFrame()

In [44]:
# tạo tập train test (chia data_train (gộp X_train và y_train) và X_test và y_test)
def splitTrainTest(data, target, ratio = 0.25): # data --> frame
    from sklearn.model_selection import train_test_split
    data_X = data.drop([target], axis = 1)
    data_y = data[[target]]
    X_train, X_test, y_train, y_test = train_test_split(data_X, data_y, test_size=ratio, random_state=42)
    data_train = X_train.copy()
    data_train[target] = y_train.reset_index(drop=True)
    return data_train, X_test, y_test # đều là dạng frame

In [45]:
# hàm tính trung bình của từng lớp trong biến target
def mean_class(data_train, target): # tên cột target, data_train là dạng pandas
    df_group = data_train.groupby(by = target).mean() # tất cả các cột đều dạng số, --> frame # sắp xếp theo bảng chữ cái tăng dần(mặc định)
    return df_group # kết quả là dataframe

In [46]:
# hàm dự đoán dùng khoảng cách euclid
def target_pred(data_group, data_test): # data_test ở dạng mảng, data_group là đã đem tính trung bình các lớp(là df_group)
    dict_ = dict()
    for index, value in enumerate(data_group.values):
        result = np.sqrt(np.sum(((data_test - value)**2), axis = 1)) # khoảng cách euclid
        if index in dict_:
            dict_.append(result)
        else:
            dict_[index] = result # Lưu ý chỗ này không phải là [result] vì result là dạng mảng, nếu thêm vào vậy thì nó chỉ có một phần tử.
    # dict_ kết quả dạng {0: [2.0], 1: [1.0540925533894596]}
    dict_[index] = result

    # Chuyển dict thành DataFrame để tìm chỉ số nhỏ nhất
    df = pd.DataFrame(dict_)
    return df.idxmin(axis = 1) # hàm này tìm cột chưá giá trị nhỏ nhất

##### Có thể phát triển: cho thêm một tham số metric vào hàm, nếu là euclid thì dùng khoảng cách euclid, manhattan thì dùng khoảng cách manhattan.

In [47]:
### Demo bằng ví dụ Iris

In [48]:
data = loadExcel('/content/Iris.xls')
data

,sepallength,sepalwidth,petallength,petalwidth,iris
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


![image.png](attachment:image.png)

In [49]:
data_train, X_test, y_test = splitTrainTest(data, 'iris', ratio = 0.3)
print(data_train)
print(X_test)
print(y_test)

     sepallength  sepalwidth  petallength  petalwidth             iris
81           5.5         2.4          3.7         1.0   Iris-virginica
133          6.3         2.8          5.1         1.5              NaN
137          6.4         3.1          5.5         1.8              NaN
75           6.6         3.0          4.4         1.4  Iris-versicolor
109          7.2         3.6          6.1         2.5              NaN
..           ...         ...          ...         ...              ...
71           6.1         2.8          4.0         1.3  Iris-versicolor
106          4.9         2.5          4.5         1.7              NaN
14           5.8         4.0          1.2         0.2  Iris-versicolor
92           5.8         2.6          4.0         1.2   Iris-virginica
102          7.1         3.0          5.9         2.1      Iris-setosa

[105 rows x 5 columns]
     sepallength  sepalwidth  petallength  petalwidth
73           6.1         2.8          4.7         1.2
18           5.7

![image.png](attachment:image.png)

In [50]:
df_group = mean_class(data_train, 'iris')
df_group

,sepallength,sepalwidth,petallength,petalwidth
iris,,,,
Iris-setosa,5.468421,2.984211,2.973684,0.842105
Iris-versicolor,5.574074,3.125926,3.162963,0.885185
Iris-virginica,5.457692,2.915385,3.153846,0.930769


![image.png](attachment:image.png)

In [51]:
#  tính khoảng cách và trả về kết quả lớp có khoảng cách gần nhất
df1 = pd.DataFrame(target_pred(df_group, X_test.values), columns = ['Predict'])
df1

,Predict
0,1
1,0
2,1
3,1
4,1
5,0
6,2
7,1
8,2
9,2


In [52]:
# set index y_test để nối 2 frame
y_test.index = range(0, len(y_test))
y_test.columns = ['Actual']
y_test

,Actual
0,Iris-versicolor
1,Iris-setosa
2,Iris-virginica
3,Iris-versicolor
4,Iris-versicolor
5,Iris-setosa
6,Iris-versicolor
7,Iris-virginica
8,Iris-versicolor
9,Iris-versicolor


In [53]:
df2 = pd.DataFrame(y_test)
df2

,Actual
0,Iris-versicolor
1,Iris-setosa
2,Iris-virginica
3,Iris-versicolor
4,Iris-versicolor
5,Iris-setosa
6,Iris-versicolor
7,Iris-virginica
8,Iris-versicolor
9,Iris-versicolor


In [54]:
pd.concat([df1, df2],axis = 1)

,Predict,Actual
0,1,Iris-versicolor
1,0,Iris-setosa
2,1,Iris-virginica
3,1,Iris-versicolor
4,1,Iris-versicolor
5,0,Iris-setosa
6,2,Iris-versicolor
7,1,Iris-virginica
8,2,Iris-versicolor
9,2,Iris-versicolor


![image.png](attachment:image.png)